# **Normalizzazione dei Nomi Aziendali**

## **Introduzione**

Nel processo di deduplicazione e record linkage, è essenziale normalizzare i nomi aziendali per evitare variazioni dovute a abbreviazioni, formati o errori di battitura. In questa fase, implementiamo una strategia per rendere i nomi uniformi e comparabili.

---

## **Step 1: Importazione delle Librerie e del DataFrame**

In [229]:
import re
import pandas as pd
import unidecode

AZIENDE_CSV = '../aziende.csv'
IMPIEGATI_CSV = '../impiegati.csv'
companies_df = pd.read_csv(AZIENDE_CSV)

/tmp/ipykernel_85681/3147525914.py:7: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  companies_df = pd.read_csv(AZIENDE_CSV)


---

## **Step 2: Definizione della Mappa di Sostituzione**
Utilizziamo un dizionario per standardizzare suffissi comuni nei nomi aziendali.
### **Questo va sicuramente migliorato magari passando tutta la prima colonna a chat**

In [230]:
REPLACEMENTS = {
    r"&": "",
    r"(?<=\s|,|\.)inc\.?\b": "",
    r"(?<=\s|,|\.)ltd\b": "",
    r"(?<=\s|,|\.)llc\b": "",
    r"(?<=\s|,|\.)plc\b": "",
    r"(?<=\s|,|\.)co\.?\b": "",  # Handles "co." and "co,"
    r"(?<=\s|,|\.)corp\.?\b": "",
    r"(?<=\s|,|\.)srl\b": "",
    r"(?<=\s|,|\.)spa\b": "",
    r"(?<=\s|,|\.)ag\b": "",
    r"(?<=\s|,|\.)sa\b": "",
    r"(?<=\s|,|\.)ab\b": "",
    r"(?<=\s|,|\.)ou\b": "",
    r"(?<=\s|,|\.)as\b": "",
    r"(?<=\s|,|\.)nv\b": "",
    r"(?<=\s|,|\.)gmbh\b": "",
    r"(?<=\s|,|\.)holding\b": "",
    r"(?<=\s|,|\.)holdings\b": "",
    r"(?<=\s|,|\.)corporation\b": "",
    r"(?<=\s|,|\.)industries\b": "",
    r"(?<=\s|,|\.)company limited\b": "",
    r"(?<=\s|,|\.)technology\b": "",
    r"(?<=\s|,|\.)solutions\b": "",
    r"(?<=\s|,|\.)systems\b": "",
    r"(?<=\s|,|\.)services\b": "",
    r"(?<=\s|,|\.)public company\b": "",
    r"(?<=\s|,|\.)limited\b": "",
    r"(?<=\s|,|\.)group\b": "",
    r"(?<=\s|,|\.)company\b": "",

    # New patterns to remove dotted abbreviations
    r"(?<=\s|,|\.)s\.?p\.?a\.?\b": "",  # Handles "S.p.A." and variations
    r"(?<=\s|,|\.)s\.?r\.?l\.?\b": "",  # Handles "S.R.L."
    r"(?<=\s|,|\.)s\.?a\.?\b": "",      # Handles "S.A."
    r"(?<=\s|,|\.)n\.?v\.?\b": ""       # Handles "N.V."
}

---

## **Step 3: Funzione di Normalizzazione**

Questa funzione applica una serie di trasformazioni per ottenere nomi uniformi:

- Conversione in minuscolo
- Rimozione di accenti
- Eliminazione di caratteri speciali
- Sostituzione dei suffissi tramite `REPLACEMENTS`

In [231]:
def normalize_name(name):
    """Normalizza i nomi delle aziende."""
    if pd.isna(name) or not isinstance(name, str):
        return ""
    
    # 1. Rimuove accenti e caratteri speciali
    name = unidecode.unidecode(name)
    
    # 2. Minuscolo e rimozione spazi extra
    name = name.lower().strip()
    
    # 3. Rimuove testo tra virgolette
    name = re.sub(r'"([^"]+)"','', name) # 3. Rimuove testo tra virgolette
    
    # 4. Applica le sostituzioni dalla mappa REPLACEMENTS
    for pattern, replacement in REPLACEMENTS.items():
        name = re.sub(pattern, replacement, name)
    
    # 5. Pulisce caratteri inutili
    name = re.sub(r"\s*,+\s*", " ", name)
    name = re.sub(r"(?<!\w)\.+(?!\w)", " ", name)
    
    # 6. Sostituisce spazi multipli con uno solo
    name = re.sub(r'\s+', ' ', name)  
    
    # 7. Eliminare punteggiatura indesiderata alla fine 
    name = re.sub(r"[.,\-]+$", "", name)
    
    return name.strip()

---

## **Step 4: Applicazione alla Colonna 'company_name'**

Eseguiamo la normalizzazione sulla colonna contenente i nomi aziendali.

In [232]:
companies_df['normalized_name'] = companies_df['company_name'].apply(normalize_name)
companies_df['modified'] = companies_df['normalized_name'] != companies_df['company_name']

---

## **Step 5: Visualizzazione e salvataggio del Risultato**

Mostriamo i primi risultati per verificare la trasformazione
e salva tutto in un csv

In [233]:
num_modified = companies_df['modified'].sum()
total_entries = len(companies_df)
percentage_modified = (num_modified / total_entries) * 100

print(f"Entries modificate: {num_modified} su {total_entries} ({percentage_modified:.2f}%)")

modified_entries = companies_df[companies_df['modified']][['company_name', 'normalized_name']]
display(modified_entries.sample(25))

companies_df = companies_df.sort_values(by=['normalized_name'])
companies_df[['company_name', 'normalized_name', 'modified']].to_csv('normalized_names.csv', index=False)

Entries modificate: 44141 su 76808 (57.47%)


,company_name,normalized_name
75026,z ltd,z
24156,food & tipple ltd,food tipple
18665,digital & you,digital you
39524,luminor pensions estonia as,luminor pensions estonia
70392,v & m management services ltd,v m management
13543,chevron u.s.a. inc.,chevron u.
64153,t a & robinson ltd,t a robinson
48229,oãœsparanrt(11905454),oaoesparanrt(11905454)
63900,t & a autos limited,t a autos
68040,treves italia srl,treves italia


---

# **Normalizzazione dei paesi**

In [234]:
filtered_df = companies_df[companies_df['country'].notna() & (companies_df['country'].str.strip() != "")].copy()
display(filtered_df[['company_name', 'country']].sample(25))

,company_name,country
6444,"autonation, inc.",united states
44417,neogen corporation,usa
64773,taiyuan heavy industry,china
67647,toyota material handling italia srl,italy
31248,hubspot,usa
40677,maruha nichiro corporation,japan
28578,guomai technologies,china
31015,howden italia spa,italy
11901,cambrian biopharma,united states
17853,db cargo italy srl,italy


In [235]:
iva_country_df = filtered_df[filtered_df['country'].astype(str).str.contains(r"\d", na=False)].copy()
display(iva_country_df[['company_name', 'country']])

,company_name,country
79,2i rete gas spa,it06724610966
85,2v energy srl,it03795470248
294,a. menarini industrie farmaceutiche riunite srl,it00395270481
303,a.i.a. agricola italiana alimentare spa,it00233470236
319,a.r.g.o. spa,it01327400352
...,...,...
75075,zanetti spa,it00373690163
75193,zeor finanziaria spa,it05678591008
75502,zignago holding spa,it03781170281
75659,zucchetti group spa,it04171890157


## **Step 1: Eliminazione delle partite IVA dai paesi**

Le partite iva sono codici da 11 cifre e possono essere precedute da due lettere iniziali rappresentanti il paese (e.g. "it") 

In [236]:
def remove_piva_from_country(country):
    if pd.isna(country) or not isinstance(country, str):
        return country
    vat_regex = r"(?:IT|it)?\d{11}"
    if re.fullmatch(vat_regex, country):
        return "italy"
    return country

---

## **Step 2: Traduzione dei nomi dei paesi in inglese**

Eliminazione di caratteri particolari come parentesi e virgolette dal campo country

In [237]:
def clean_country(country): 
    if pd.isna(country) or not isinstance(country, str):
        return ""
    
    # Rimuove parentesi tonde ( ), quadre [ ] e singole virgolette '
    country = re.sub(r"[\[\]']", "", country)  
    country = re.sub(r"\s*\(.*?\)\s*", "", country).strip()  
        
    return country

In [238]:
import pycountry

country_mapping = {
    'it': 'italy',            # Codici e abbreviazioni per l'Italia
    'uk': 'united kingdom',   # Abbreviazione per il Regno Unito
    'usa': 'united states',   # Abbreviazione per gli Stati Uniti
    'us': 'united states',    # Abbreviazione per gli Stati Uniti (altro formato)
    'fr': 'france',           # Abbreviazione per la Francia
    'de': 'germany',          # Abbreviazione per la Germania
    'es': 'spain',            # Abbreviazione per la Spagna
    'ca': 'canada',           # Abbreviazione per il Canada
    'au': 'australia',        # Abbreviazione per l'Australia
    'cn': 'china',            # Abbreviazione per la Cina
    'in': 'india',            # Abbreviazione per l'India
    'jp': 'japan',            # Abbreviazione per il Giappone
    'br': 'brazil',           # Abbreviazione per il Brasile
    'kr': 'south korea',      # Abbreviazione per la Corea del Sud
    'mx': 'mexico',           # Abbreviazione per il Messico
    'se': 'sweden',           # Abbreviazione per la Svezia
    'no': 'norway',           # Abbreviazione per la Norvegia
    'fi': 'finland',          # Abbreviazione per la Finlandia
    'nl': 'netherlands',      # Abbreviazione per i Paesi Bassi
    'ch': 'switzerland',      # Abbreviazione per la Svizzera
    'ru': 'russia',           # Abbreviazione per la Russia
    'sa': 'south africa',     # Abbreviazione per il Sudafrica
    'kr': 'south korea',      # Abbreviazione per la Corea del Sud
    'sg': 'singapore',        # Abbreviazione per Singapore
    "uae": "united Arab Emirates",
}

In [239]:
def translate_country(country):
    if pd.isna(country) or not isinstance(country, str) or country.strip() == "":
        return ""
    
    if country in country_mapping:
        return country_mapping[country]
    
    try:
        country_obj = pycountry.countries.lookup(country)
        return country_obj.name
    except LookupError:
        return country

In [240]:
def normalize_country(country):
    if pd.isna(country) or not isinstance(country, str) or country.strip() == "":
        return ""
    
    # Se ci sono più paesi separati da virgole, li normalizziamo singolarmente
    country_list = [c.lower().strip() for c in country.split(",")]

    normalized_countries = []
    for c in country_list:
        c = clean_country(c)
        c = remove_piva_from_country(c)
        c = translate_country(c)
        normalized_countries.append(c.lower().strip())
    
    return ", ".join(normalized_countries)

In [241]:
companies_df['normalized_country'] = companies_df['country'].apply(normalize_country)
companies_df['modified'] = companies_df['normalized_country'] != companies_df['country']

In [242]:
# Exclude NaN to "" modifications
valid_modifications = companies_df[
    (companies_df["modified"]) & ~(companies_df["country"].isna() & (companies_df["normalized_country"] == ""))
]

num_modified = len(valid_modifications)
total_entries = len(companies_df)
percentage_modified = (num_modified / total_entries) * 100

print(f"Entries modificate: {num_modified} su {total_entries} ({percentage_modified:.2f}%)")

modified_entries = valid_modifications[['country', 'normalized_country']]
display(modified_entries.sample(25))

companies_df = companies_df.sort_values(by=['normalized_country'])
companies_df[['country', 'normalized_country', 'modified']].to_csv('normalized_country.csv', index=False)

Entries modificate: 8258 su 76808 (10.75%)


,country,normalized_country
61238,usa,united states
42393,uk,united kingdom
8087,usa,united states
28227,it00169710241,italy
6013,usa,united states
10056,usa,united states
2528,usa,united states
5563,uk,united kingdom
19007,usa,united states
65541,uk,united kingdom


## Eliminazione duplicati nello schema mediato nel file aziende_normalizzate.csv


In [ ]:
import pandas as pd


aziende_df = pd.read_csv("../aziende_normalizzate.csv", encoding="utf-8")


# Conta i duplicati nel file delle aziende
aziende_duplicati = aziende_df.duplicated().sum()

# Trova le righe duplicate
aziende_duplicati_righe = aziende_df[aziende_df.duplicated()]

# Stampa i risultati
print(f"Numero di righe duplicate nel file 'aziende_normalizzate.csv': {aziende_duplicati}")

# Mostra le prime righe duplicate (se ci sono)
if aziende_duplicati > 0:
    print("\nEsempio di righe duplicate nel file 'aziende_normalizzate.csv':")
    print(aziende_duplicati_righe.head())
else:
    print("\nNon ci sono righe duplicate nel file.")


C:\Users\emigi\AppData\Local\Temp\ipykernel_27024\3067150653.py:4: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  aziende_df = pd.read_csv("../aziende_normalizzate.csv", encoding="utf-8")


Numero di righe duplicate nel file 'aziende_normalizzate.csv': 1482

Esempio di righe duplicate nel file 'aziende_normalizzate.csv':
     company_id             company_name trade_name  industry sector  \
1019   02921121  'q' accountancy limited        NaN       NaN    NaN   
1210   12816840         5d uk operations        NaN       NaN    NaN   
1212   12816020        5d uk real estate        NaN       NaN    NaN   
1226        NaN          6d technologies        NaN   telecom    NaN   
1268        NaN                  99acres        NaN  internet    NaN   

     categories company_status             company_type  \
1019        NaN         active  private limited company   
1210        NaN         active  private limited company   
1212        NaN         active  private limited company   
1226        NaN            NaN                      NaN   
1268        NaN            NaN                   public   

                               headquarters  \
1019                            

In [ ]:
# Elimina i duplicati
aziende_df_senza_duplicati = aziende_df.drop_duplicates()

# Salva il DataFrame senza duplicati in un nuovo file CSV
aziende_df_senza_duplicati.to_csv("aziende_senza_duplicati.csv", index=False)

# Visualizza un'anteprima dei dati senza duplicati
print(aziende_df_senza_duplicati.head())

  company_id company_name trade_name  \
0       1470          NaN        NaN   
1          7          NaN        NaN   
2          8          NaN        NaN   
3          9          NaN        NaN   
4         11          NaN        NaN   

                                            industry sector categories  \
0   other business support service activities n.e.c.    NaN        NaN   
1      manufacture of plastic packing goods \xc2\xa0    NaN        NaN   
2  other retail sale not in stores, stalls or mar...    NaN        NaN   
3  activities of saunas, sunbeds and massage salo...    NaN        NaN   
4  business and other management consultancy acti...    NaN        NaN   

  company_status company_type headquarters address  ... nace_code facebook  \
0            NaN          NaN          NaN     NaN  ...     82.99      NaN   
1            NaN          NaN          NaN     NaN  ...     22.22      NaN   
2            NaN          NaN          NaN     NaN  ...     47.99      NaN   
3 

# **Normalizzazione delle città**

In [243]:
filtered_df = companies_df[companies_df['city'].notna() & (companies_df['city'].str.strip() != "")].copy()
display(filtered_df[['company_name', 'city']].sample(25))

,company_name,city
58954,shift technology,paris
28303,gtt italy srl,roma
75090,"zara usa, inc.",new york
5066,arco vara as,tallinn
45601,nippon life insurance company of america,new york
40798,masterplast italia srl,bibbiano (barco )
10554,bonatti spa,parma
34356,j.p. morgan trust company (jersey) limited,jersey
18517,dhl supply chain limited,milton keynes
5309,arper spa,monastier di treviso


### **Step 1: Eliminare eventuale testo tra parentesi** 
La cosa che sempra più intuitiva da fare è eliminare eventuale testo tra parentesi dopo la città (perdendo un po' di informazione, ma comunque accettabile)

In [244]:
def clean_city(city):
    if pd.isna(city) or not isinstance(city, str):
        return ""
    
    city = re.sub(r"\s*\(.*?\)\s*", "", city).strip()
    return city 

### **Step 2: Tradurre nomi di città italiane in italiano** 

Bisognerebbe anche indagare se qualche città viene riportata in lingue diverse...

In [12]:
# Query in italiano
milano_df = filtered_df[filtered_df["city"] == "milano"]
roma_df = filtered_df[filtered_df["city"] == "roma"]

# Query in inglese
milan_df = filtered_df[filtered_df["city"] == "milan"]
rome_df = filtered_df[filtered_df["city"] == "rome"]

In [13]:
display(milano_df[['company_name', 'city']])
display(milan_df[['company_name', 'city']])

,company_name,city
562,aberdeen standard investments ireland limited,milano
622,above comparison,milano
623,above comparison,milano
766,accenture,milano
765,accenture,milano
...,...,...
73454,withsecure,milano
74336,yes ticket,milano
74335,yes ticket,milano
75694,zurich italy bank,milano


,company_name,city
57140,satispay,milan
57364,scalapay,milan


In [14]:
display(roma_df[['company_name', 'city']])
display(rome_df[['company_name', 'city']])

,company_name,city
1431,aec underwriting agenzia di assicurazione e ri...,roma
1430,aec underwriting agenzia di assicurazione e ri...,roma
1739,agenzia delle entrate,roma
2463,ald automotive italia,roma
4282,angelini pharma italia aziende chimiche riunit...,roma
...,...,...
68023,trenitalia,roma
69746,universita' degli studi di roma la sapienza,roma
69748,universita' degli studi roma tre,roma
73045,western union payment ireland limited,roma


,company_name,city


Probabilmente, visto la sorgente dati, le aziende italiane sono state estratte da repository italiani o almeno convertite con città in italiano. Solo due aziende (*satispay* e *scalapay*) hanno il nome riportato in inglese.

Le città italiane sono segnate maggiormente in italiano possiamo pensare di **convertire le città con nome inglese in italiano** 

Abbiamo bisogno di un dizionario che mappi i nomi delle città (City-mapping). Evitiamo di scriverlo da zero e ne usiamo uno noto da qualche libreria (*do not reinvent the wheel...*).

Usiamo dunque la libreria `geopy` che con **GeoNames**, un database globale di località, ci permette di convertire i nomi 

In [279]:
from geopy.geocoders import Nominatim
import time
from tqdm import tqdm
import pickle

CITY_CACHE_FILE = "geopy_city_cache.pkl"
tqdm.pandas()
geolocator = Nominatim(user_agent="city_normalizer")

try:
    with open(CITY_CACHE_FILE, 'rb') as f:
        city_cache = pickle.load(f)
    print("Found saved cache...")
except FileNotFoundError:
    city_cache = {}
    print("Cache file not found, creating a new one...")

cache_hit = 0
cache_miss = 0

Found saved cache...


In [280]:
def normalize_city(city):
    global cache_hit, cache_miss
    city = clean_city(city)

    if city in city_cache:
        cache_hit += 1
        return city_cache[city]
    
    cache_miss += 1
    
    try:
        time.sleep(0.5)
        location = geolocator.geocode(city, exactly_one=True, timeout=10)
        if location and "Italia" in location.address:
            location_it = geolocator.geocode(city, language="it", timeout=10)
            if location_it:
                it_city = location_it.raw.get("display_name", city).split(",")[0].lower().strip()
                city_cache[city] = it_city
                return it_city
    except Exception as e:
        print(f"Errore con {city}: {e}")

    city_cache[city] = city
    return city

In [287]:
def tqdm_wrapper(row, tqdm_bar):
    if pd.isna(row['city']) or not isinstance(row['city'], str) or row['city'] == "":
        tqdm_bar.update(1)
        return ""    
    
    global cache_miss, cache_hit
    normalized = normalize_city(row['city']) if row['country'] == "italy" else row['city']
    total_req = cache_hit + cache_miss
    hit_ratio = (cache_hit / total_req) * 100 if total_req > 0 else 0
    tqdm_bar.set_description(f"Processing | Cache Hit Ratio: {hit_ratio:.2f}%")  # Update bar without new lines
    tqdm_bar.update(1)

    return normalized

**GeoNames** funziona, adesso applichiamolo all'intero dataframe

In [288]:
with tqdm(total=len(companies_df), desc="Processing", unit="entry") as tqdm_bar:
        companies_df['city_translate'] = companies_df.apply(lambda row: tqdm_wrapper(row, tqdm_bar), axis=1)

with open(CITY_CACHE_FILE, 'wb') as f:
        pickle.dump(city_cache, f)
        
companies_df['modified'] = companies_df['city'] != companies_df['city_translate']

Processing | Cache Hit Ratio: 100.00%: 100%|██████████| 76808/76808 [00:10<00:00, 6997.72entry/s] 


In [293]:
valid_modifications = companies_df[
    (companies_df["modified"]) & ~(companies_df["city"].isna() & (companies_df["city_translate"] == ""))
]

num_modified = len(valid_modifications)
total_cities = len(companies_df)
percentage_modified = (num_modified / total_cities) * 100

print(f"Città tradotte: {num_modified} su {total_cities} ({percentage_modified:.2f}%)")

translated_cities = valid_modifications[['normalized_name', 'city', 'city_translate']]
display(translated_cities.sample(25))

companies_df = companies_df.sort_values(by=['city'])
companies_df[['normalized_name', 'city', 'city_translate']].to_csv('translated_cities.csv', index=False)

Città tradotte: 170 su 76808 (0.22%)


,normalized_name,city,city_translate
34161,itnet,"assago (milanofiori nord, palazzo u4 )",assago
57629,scuderia ferrari club scarl,maranello (c/o ferrari spa ),maranello
42036,microbial control italy,segrate (segreen business park pal. y ),segrate
25034,friulsider,san giovanni al natisone (villanova del judrio ),san giovanni al natisone
21872,esseco,trecate (san martino ),trecate
37765,lactalis ingredients italia,torrile (san polo ),torrile
39039,lloyd's register emea,genova (x piano ),genova
56301,s.i.a.t.societa' italiana acciai trafilati,osoppo (c/o uffici della ferriere nord ),osoppo
22737,fameccanica.data,san giovanni teatino (sambuceto ),san giovanni teatino
73457,witor's,corte de' frati (aspice ),corte de' frati


---

### **Salvataggio delle modifiche sul csv**

In [294]:
companies_df['country'] = companies_df['normalized_country']
companies_df['company_name'] = companies_df['normalized_name']
companies_df['city'] = companies_df['city_translate']

companies_df.drop(columns=['normalized_name', 'normalized_country', 'city_translate', 'modified'])
companies_df = companies_df.sort_values(by=['company_name'])
companies_df.to_csv('../aziende_normalizzate.csv', index=False)